In [1]:
from contextlib import closing
import os
import platform
import socket
import subprocess
from subprocess import PIPE
import tempfile

from py4j.java_gateway import JavaGateway, GatewayParameters
from py4j.java_collections import ListConverter

In [24]:
__ERODE_JAR__ = "erodeMetrics.jar"#os.path.join(os.path.dirname(__file__), "erodeMetrics.jar")
def _start_server():
    # find a free port
    for port in range(25333, 65545):
        with closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as s:
            dest_addr = ("127.0.0.1", port)
            if s.connect_ex(dest_addr):
                break

    #ld_path = __ERODE_LIB_DIR__
    #java="java"
    java ="/Library/Java/JavaVirtualMachines/jdk-11.0.14.jdk/Contents/Home/bin/java"
    argv = [java, #f'-Djava.library.path="{ld_path}"',
                "-jar", __ERODE_JAR__, str(port)]

    if platform.system() == "Linux":
        env_ld_path = os.getenv("LD_LIBRARY_PATH")
        if env_ld_path:
            ld_path = f"{ld_path}:{env_ld_path}"
        env ={"LD_LIBRARY_PATH": ld_path}
        proc = subprocess.Popen(" ".join(argv), stdout=PIPE,
                shell=True, env=env)
    else:
        proc = subprocess.Popen(argv, stdout=PIPE)
    proc.stdout.readline()
    return proc, port

def _stop_server(proc):
    proc.terminate()
    try:
        proc.wait(5)
    except subprocess.TimeoutExpired:
        proc.kill()
        proc.wait()

In [25]:
_proc, _port = _start_server()
gw_params = GatewayParameters(port=_port)#, auto_convert=True)
_gw = JavaGateway(gateway_parameters=gw_params)
erode = _gw.entry_point

In [26]:
_proc

<Popen: returncode: None args: ['/Library/Java/JavaVirtualMachines/jdk-11.0....>

In [27]:
erode.loadModel("AM.ode");
obtained = erode.computeBB();
obtained
print(erode.getPartitionString(obtained))

a=erode.computeMetrics()
for s in a:
    print(s)

The partition has 1 block out of 3 species:
Block 1, Size: 3
0-x0 
1-x1 
2-x2 


Solution:
Objective value = 3.9
x = 1.0
y = 0.9



In [17]:
_port

25334

In [23]:
_stop_server(_proc)